In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from collections import Counter
import xgboost as xgb


In [3]:
# Cargar el dataset
vinotinto_df = pd.read_csv('../data/processed/vinotinto.csv')

# Convertir quality_label a valores numéricos
label_encoder = LabelEncoder()
vinotinto_df['quality_label_encoded'] = label_encoder.fit_transform(vinotinto_df['quality_label'])

# Definir características (X) y variable objetivo (y)
X = vinotinto_df.drop(['quality', 'quality_label', 'quality_label_encoded'], axis=1)
y = vinotinto_df['quality_label_encoded']


In [4]:

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [5]:

# Balanceo de clases con SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Ver el balance de clases después de SMOTE
print(f"Antes de SMOTE: {Counter(y_train)}")
print(f"Después de SMOTE: {Counter(y_res)}")


Antes de SMOTE: Counter({1: 448, 2: 374, 0: 129})
Después de SMOTE: Counter({2: 448, 1: 448, 0: 448})


In [ ]:

from sklearn.model_selection import GridSearchCV

# Definir los parámetros a probar para cada modelo

# Regresión Logística
param_grid_logreg = {'C': [0.1, 1, 10, 20, 100], 'solver': ['lbfgs', 'liblinear']}
logreg = LogisticRegression(class_weight='balanced', max_iter=10000)
grid_search_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5, scoring='accuracy')
grid_search_logreg.fit(X_res, y_res)
print("Mejores parámetros para Regresión Logística:", grid_search_logreg.best_params_)

# Árbol de Decisión
param_grid_dtree = {'max_depth': [3, 5, 10, 20, None], 'min_samples_split': [2, 10, 20, 40]}
dtree = DecisionTreeClassifier(class_weight='balanced')
grid_search_dtree = GridSearchCV(dtree, param_grid_dtree, cv=5, scoring='accuracy')
grid_search_dtree.fit(X_res, y_res)
print("Mejores parámetros para Árbol de Decisión:", grid_search_dtree.best_params_)

# Random Forest
param_grid_rf = {'n_estimators': [50, 100, 200, 500, 1000], 'max_depth': [3, 5, 10, 20, None]}
rf = RandomForestClassifier(class_weight='balanced')
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy')
grid_search_rf.fit(X_res, y_res)
print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

# SVM
param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
svm = SVC(class_weight='balanced')
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_res, y_res)
print("Mejores parámetros para SVM:", grid_search_svm.best_params_)

# KNN
param_grid_knn = {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='accuracy')
grid_search_knn.fit(X_res, y_res)
print("Mejores parámetros para KNN:", grid_search_knn.best_params_)


In [ ]:

# Usar los mejores modelos encontrados en GridSearchCV
best_logreg = grid_search_logreg.best_estimator_
best_dtree = grid_search_dtree.best_estimator_
best_rf = grid_search_rf.best_estimator_
best_svm = grid_search_svm.best_estimator_
best_knn = grid_search_knn.best_estimator_

# Entrenar con los datos balanceados por SMOTE
best_logreg.fit(X_res, y_res)
best_dtree.fit(X_res, y_res)
best_rf.fit(X_res, y_res)
best_svm.fit(X_res, y_res)
best_knn.fit(X_res, y_res)

# Predecir en el conjunto de prueba
y_pred_logreg = best_logreg.predict(X_test)
y_pred_dtree = best_dtree.predict(X_test)
y_pred_rf = best_rf.predict(X_test)
y_pred_svm = best_svm.predict(X_test)
y_pred_knn = best_knn.predict(X_test)

# Evaluar el rendimiento de cada modelo
from sklearn.metrics import classification_report

print("Evaluación Regresión Logística:")
print(classification_report(y_test, y_pred_logreg))

print("Evaluación Árbol de Decisión:")
print(classification_report(y_test, y_pred_dtree))

print("Evaluación Random Forest:")
print(classification_report(y_test, y_pred_rf))

print("Evaluación SVM:")
print(classification_report(y_test, y_pred_svm))

print("Evaluación KNN:")
print(classification_report(y_test, y_pred_knn))


Evaluación Regresión Logística:
              precision    recall  f1-score   support

           0       0.49      0.78      0.60        55
           1       0.68      0.71      0.70       192
           2       0.54      0.40      0.46       161

    accuracy                           0.60       408
   macro avg       0.57      0.63      0.59       408
weighted avg       0.60      0.60      0.59       408

Evaluación Árbol de Decisión:
              precision    recall  f1-score   support

           0       0.37      0.42      0.39        55
           1       0.70      0.66      0.68       192
           2       0.50      0.51      0.51       161

    accuracy                           0.57       408
   macro avg       0.52      0.53      0.53       408
weighted avg       0.58      0.57      0.57       408

Evaluación Random Forest:
              precision    recall  f1-score   support

           0       0.51      0.64      0.56        55
           1       0.72      0.69      0.

In [ ]:

# Calcular el valor de scale_pos_weight para XGBoost
scale_pos_weight = len(y_res) / (3 * np.bincount(y_res))

# Inicializar y entrenar el modelo XGBoost con scale_pos_weight
xgb_model = xgb.XGBClassifier(scale_pos_weight=scale_pos_weight, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_res, y_res)

# Predecir en el conjunto de prueba
y_pred_xgb = xgb_model.predict(X_test)

# Evaluar el rendimiento
print("Evaluación XGBoost:")
print(classification_report(y_test, y_pred_xgb))


c:\Users\inaki\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:30:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Evaluación XGBoost:
              precision    recall  f1-score   support

           0       0.48      0.56      0.52        55
           1       0.70      0.68      0.69       192
           2       0.49      0.48      0.48       161

    accuracy                           0.59       408
   macro avg       0.56      0.57      0.57       408
weighted avg       0.59      0.59      0.59       408



utilizando Pesos de clase

In [ ]:
# Entrenar los modelos utilizando el parámetro `class_weight='balanced'`

# Random Forest con pesos de clase
rf_balanced = RandomForestClassifier(class_weight='balanced', n_estimators=1000, max_depth=20, random_state=42)
rf_balanced.fit(X_train, y_train)

# Predicción y evaluación
y_pred_rf_balanced = rf_balanced.predict(X_test)
print("Evaluación Random Forest con Pesos de Clase:")
print(classification_report(y_test, y_pred_rf_balanced))

# Otros modelos con pesos de clase

# Regresión Logística
logreg_balanced = LogisticRegression(class_weight='balanced', C=100, solver='lbfgs', max_iter=10000)
logreg_balanced.fit(X_train, y_train)
y_pred_logreg_balanced = logreg_balanced.predict(X_test)
print("Evaluación Regresión Logística con Pesos de Clase:")
print(classification_report(y_test, y_pred_logreg_balanced))

# Árbol de Decisión
dtree_balanced = DecisionTreeClassifier(class_weight='balanced', max_depth=None, min_samples_split=2)
dtree_balanced.fit(X_train, y_train)
y_pred_dtree_balanced = dtree_balanced.predict(X_test)
print("Evaluación Árbol de Decisión con Pesos de Clase:")
print(classification_report(y_test, y_pred_dtree_balanced))

# SVM
svm_balanced = SVC(class_weight='balanced', C=10, gamma='auto', kernel='rbf')
svm_balanced.fit(X_train, y_train)
y_pred_svm_balanced = svm_balanced.predict(X_test)
print("Evaluación SVM con Pesos de Clase:")
print(classification_report(y_test, y_pred_svm_balanced))

# KNN
knn_balanced = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn_balanced.fit(X_train, y_train)
y_pred_knn_balanced = knn_balanced.predict(X_test)
print("Evaluación KNN con Pesos de Clase:")
print(classification_report(y_test, y_pred_knn_balanced))


Evaluación Random Forest con Pesos de Clase:
              precision    recall  f1-score   support

           0       0.69      0.33      0.44        55
           1       0.70      0.71      0.70       192
           2       0.52      0.61      0.56       161

    accuracy                           0.62       408
   macro avg       0.64      0.55      0.57       408
weighted avg       0.63      0.62      0.61       408

Evaluación Regresión Logística con Pesos de Clase:
              precision    recall  f1-score   support

           0       0.47      0.80      0.59        55
           1       0.68      0.71      0.69       192
           2       0.53      0.38      0.44       161

    accuracy                           0.59       408
   macro avg       0.56      0.63      0.58       408
weighted avg       0.59      0.59      0.58       408

Evaluación Árbol de Decisión con Pesos de Clase:
              precision    recall  f1-score   support

           0       0.38      0.38     

El mejor modelos es el RAndomFOrest

Script para Afinar los Hiperparámetros de Random Forest:

In [ ]:
#pip install tqdm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Definir el modelo Random Forest
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Definir el rango de parámetros a probar
# param_grid_rf = {
#     'n_estimators': [100, 200, 300, 500, 1000],  # Número de árboles
#     'max_depth': [3, 5, 10, 20, None],  # Profundidad máxima de los árboles
#     'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
#     'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras en cada hoja
#     'max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar en cada división
#     'bootstrap': [True, False]  # Si se utiliza muestreo bootstrap en la construcción de árboles
# }


param_grid_rf = {
    'n_estimators': [100, 200],  # Solo probar 2 números de árboles
    'max_depth': [3, 5],  # Solo probar dos profundidades
    'min_samples_split': [2],  # Solo probar la mínima cantidad de muestras para dividir
    'min_samples_leaf': [1],  # Solo una opción
    'max_features': ['auto'],  # Una sola opción
    'bootstrap': [True]  # Solo una opción
}

# Inicializar GridSearchCV con el modelo y el rango de parámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Añadir la barra de progreso con tqdm
# Usamos tqdm para envolver la lista de combinaciones de parámetros en GridSearchCV
# Esto nos da una barra de progreso mientras realiza las combinaciones de prueba
# El número total de combinaciones de parámetros es el número de pruebas que se harán
tuned_params = list(grid_search_rf.param_grid.values())  # Extraer los parámetros a probar
total_combinations = np.prod([len(x) for x in tuned_params])  # Total de combinaciones

# Ajustar el modelo con la barra de progreso
for _ in tqdm(range(total_combinations), desc="Entrenando GridSearchCV", total=total_combinations):
    grid_search_rf.fit(X_res, y_res)

# Ver los mejores parámetros encontrados
print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

# Usar el mejor modelo encontrado en GridSearchCV
best_rf = grid_search_rf.best_estimator_

# Predecir en el conjunto de prueba
y_pred_rf = best_rf.predict(X_test)

# Evaluar el rendimiento con los mejores parámetros
from sklearn.metrics import classification_report
print("Evaluación Random Forest con los mejores parámetros:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Definir el modelo Random Forest
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Definir el rango de parámetros a probar
param_grid_rf = {
    'n_estimators': [100, 200, 300, 500, 1000],  # Número de árboles
    'max_depth': [3, 5, 10, 20, None],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras en cada hoja
    'max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar en cada división
    'bootstrap': [True, False]  # Si se utiliza muestreo bootstrap en la construcción de árboles
}

# Inicializar GridSearchCV con el modelo y el rango de parámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)

# Crear la barra de progreso
total_combinations = np.prod([len(v) for v in param_grid_rf.values()])  # Total de combinaciones
progress_bar = tqdm(total=total_combinations, desc="Entrenando GridSearchCV", position=0, leave=True)

# Crear una función de callback para actualizar la barra de progreso
def callback(*args, **kwargs):
    progress_bar.update(1)  # Actualiza la barra de progreso en cada ajuste

# Añadir la barra de progreso y callback a GridSearchCV
grid_search_rf.fit(X_res, y_res)
progress_bar.close()  # Cerrar la barra al final

# Ver los mejores parámetros encontrados
print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

# Usar el mejor modelo encontrado en GridSearchCV
best_rf = grid_search_rf.best_estimator_

# Predecir en el conjunto de prueba
y_pred_rf = best_rf.predict(X_test)

# Evaluar el rendimiento con los mejores parámetros
from sklearn.metrics import classification_report
print("Evaluación Random Forest con los mejores parámetros:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Definir el modelo Random Forest
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Definir el rango de parámetros a probar
param_grid_rf = {
    'n_estimators': [100, 200, 300, ],  # Número de árboles
    'max_depth': [3, 5, 10, 20, None],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras en cada hoja
    'max_features': ['sqrt', 'log2', None],  # Número de características a considerar en cada división
    'bootstrap': [True, False]  # Si se utiliza muestreo bootstrap en la construcción de árboles
}

# Inicializar GridSearchCV con el modelo y el rango de parámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Añadir la barra de progreso con tqdm
# Usamos tqdm para envolver la lista de combinaciones de parámetros en GridSearchCV
# Esto nos da una barra de progreso mientras realiza las combinaciones de prueba
# El número total de combinaciones de parámetros es el número de pruebas que se harán
tuned_params = list(grid_search_rf.param_grid.values())  # Extraer los parámetros a probar
total_combinations = np.prod([len(x) for x in tuned_params])  # Total de combinaciones

# Ajustar el modelo con la barra de progreso
for _ in tqdm(range(total_combinations), desc="Entrenando GridSearchCV", total=total_combinations):
    grid_search_rf.fit(X_res, y_res)

# Ver los mejores parámetros encontrados
print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)

# Usar el mejor modelo encontrado en GridSearchCV
best_rf = grid_search_rf.best_estimator_

# Predecir en el conjunto de prueba
y_pred_rf = best_rf.predict(X_test)

# Evaluar el rendimiento con los mejores parámetros
from sklearn.metrics import classification_report
print("Evaluación Random Forest con los mejores parámetros:")
print(classification_report(y_test, y_pred_rf))


Entrenando GridSearchCV:   0%|          | 0/810 [00:00<?, ?it/s]


NameError: name 'X_res' is not defined

Guardar el modelo con pickle:

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier

# Asumiendo que best_rf es el modelo que ya has ajustado con GridSearchCV

# Guardar el modelo entrenado en un archivo
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(best_rf, file)

# Para cargar el modelo y usarlo más tarde:
with open('random_forest_model.pkl', 'rb') as file:
    loaded_rf_model = pickle.load(file)

# Usar el modelo cargado para hacer predicciones
y_pred_loaded_rf = loaded_rf_model.predict(X_test)

# Evaluar el rendimiento
from sklearn.metrics import classification_report
print("Evaluación del modelo cargado:")
print(classification_report(y_test, y_pred_loaded_rf))


Guardar el modelo con TensorFlow (para redes neuronales):

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Ejemplo de creación de un modelo secuencial con TensorFlow
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Para clasificación de 3 clases
])

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_res, y_res, epochs=10, batch_size=32)

# Guardar el modelo
model.save('tensorflow_model.h5')

# Cargar el modelo
loaded_model = tf.keras.models.load_model('tensorflow_model.h5')

# Hacer predicciones con el modelo cargado
y_pred_loaded_model = loaded_model.predict(X_test)

# Convertir las probabilidades a etiquetas de clase
y_pred_class = np.argmax(y_pred_loaded_model, axis=1)

# Evaluar el rendimiento
from sklearn.metrics import classification_report
print("Evaluación del modelo cargado de TensorFlow:")
print(classification_report(y_test, y_pred_class))
